In [1]:
# On affiche les graphiques dans le notebook en statique
%matplotlib inline

In [2]:
import numpy as np
import logging
import sys
import os
import gc
import joblib
import lightgbm as lgb
import optuna
import pandas as pd
from sklearn.model_selection import cross_val_score

In [3]:
from src.p7_constantes import DATA_INTERIM, DATA_BASE, MODEL_DIR
from src.p7_util import timer

Démarrer

In [4]:
df = pd.read_csv(os.path.join(DATA_INTERIM, "train.csv"))
print("Forme de train.csv :", df.shape)
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)
df.head()

Forme de train.csv : (307506, 18)


,SK_ID_CURR,TARGET,ORGANIZATION_TYPE,CREDIT_TO_ANNUITY_RATIO,EXT_SOURCES_MEAN,EXT_SOURCE_3,EXT_SOURCES_MIN,BUREAU_CONSUMER_DAYS_CREDIT_ENDDATE_MAX,AMT_ANNUITY,CREDIT_TO_GOODS_RATIO,OCCUPATION_TYPE,DAYS_ID_PUBLISH,DAYS_EMPLOYED,BUREAU_CREDIT_DEBT_CREDIT_DIFF_MEAN,BUREAU_ACTIVE_DEBT_CREDIT_DIFF_MEAN,ANNUITY_TO_INCOME_RATIO,AGE
0,100002,1.0,0,16.47,0.1617,0.1394,0.0830,780.0,24700.5,1.1580,0,-2120,-637.0,15994.282,118103.78,0.12195,25.920548
1,100003,0.0,1,36.22,0.4668,NaN,0.3113,-420.0,35698.5,1.1455,1,-291,-1188.0,461250.000,810000.00,0.13220,45.931507
2,100004,0.0,2,20.00,0.6426,0.7295,0.5560,-382.0,6750.0,1.0000,0,-2531,-225.0,NaN,NaN,0.10000,52.180822
3,100006,0.0,0,10.53,0.6504,NaN,0.6504,NaN,29686.5,1.0530,0,-2437,-3040.0,NaN,NaN,0.21980,52.068493
4,100007,0.0,3,23.47,0.3228,NaN,0.3228,-783.0,21865.5,1.0000,1,-3458,-3038.0,NaN,NaN,0.17990,54.608219


In [5]:
encoded_features_application = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']
mapper = {feature: feature + '_old' for feature in encoded_features_application}
df = df.rename(mapper, axis=1)


In [6]:
initial_application = pd.read_csv(os.path.join(DATA_BASE, "application_train.csv"))[['SK_ID_CURR'] + encoded_features_application]
initial_application.shape
df = pd.merge(left=df, right=initial_application, on=['SK_ID_CURR'])
df = df.drop([v for v in mapper.values()], axis=1)
del initial_application
del mapper
gc.collect()
df.head()

,SK_ID_CURR,TARGET,CREDIT_TO_ANNUITY_RATIO,EXT_SOURCES_MEAN,EXT_SOURCE_3,EXT_SOURCES_MIN,BUREAU_CONSUMER_DAYS_CREDIT_ENDDATE_MAX,AMT_ANNUITY,CREDIT_TO_GOODS_RATIO,DAYS_ID_PUBLISH,DAYS_EMPLOYED,BUREAU_CREDIT_DEBT_CREDIT_DIFF_MEAN,BUREAU_ACTIVE_DEBT_CREDIT_DIFF_MEAN,ANNUITY_TO_INCOME_RATIO,AGE,ORGANIZATION_TYPE,OCCUPATION_TYPE
0,100002,1.0,16.47,0.1617,0.1394,0.0830,780.0,24700.5,1.1580,-2120,-637.0,15994.282,118103.78,0.12195,25.920548,Business Entity Type 3,Laborers
1,100003,0.0,36.22,0.4668,NaN,0.3113,-420.0,35698.5,1.1455,-291,-1188.0,461250.000,810000.00,0.13220,45.931507,School,Core staff
2,100004,0.0,20.00,0.6426,0.7295,0.5560,-382.0,6750.0,1.0000,-2531,-225.0,NaN,NaN,0.10000,52.180822,Government,Laborers
3,100006,0.0,10.53,0.6504,NaN,0.6504,NaN,29686.5,1.0530,-2437,-3040.0,NaN,NaN,0.21980,52.068493,Business Entity Type 3,Laborers
4,100007,0.0,23.47,0.3228,NaN,0.3228,-783.0,21865.5,1.0000,-3458,-3038.0,NaN,NaN,0.17990,54.608219,Religion,Core staff


In [7]:
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].apply(lambda x: np.nan if x=="XNA" else x)
df['ORGANIZATION_TYPE'].value_counts(dropna=False)

ORGANIZATION_TYPE
Business Entity Type 3    67991
NaN                       55374
Self-employed             38412
Other                     16683
Medicine                  11192
Business Entity Type 2    10553
Government                10404
School                     8893
Trade: type 7              7831
Kindergarten               6879
Construction               6721
Business Entity Type 1     5983
Transport: type 4          5398
Trade: type 3              3492
Industry: type 9           3368
Industry: type 3           3277
Security                   3247
Housing                    2958
Industry: type 11          2704
Military                   2634
Bank                       2507
Agriculture                2454
Police                     2341
Transport: type 2          2204
Postal                     2157
Security Ministries        1974
Trade: type 2              1900
Restaurant                 1811
Services                   1575
University                 1327
Industry: type 7      

In [8]:
df['OCCUPATION_TYPE'].value_counts(dropna=False)

OCCUPATION_TYPE
NaN                      96389
Laborers                 55185
Sales staff              32102
Core staff               27569
Managers                 21371
Drivers                  18603
High skill tech staff    11380
Accountants               9813
Medicine staff            8537
Security staff            6721
Cooking staff             5946
Cleaning staff            4653
Private service staff     2652
Low-skill Laborers        2092
Waiters/barmen staff      1348
Secretaries               1305
Realty agents              751
HR staff                   563
IT staff                   526
Name: count, dtype: int64

In [9]:
cat_features = list(df.loc[:, df.dtypes == 'object'].columns.values)
for feature in cat_features:
    df[feature] = pd.Series(df[feature], dtype="category")

In [10]:
# Essai sur un échantillon d'abord
n_rows = 30_000
X = df.drop(columns=["SK_ID_CURR", "TARGET"], axis=1)
y = df["TARGET"]
if n_rows:
    X = X.head(n_rows)
    y = y.head(n_rows)

In [11]:
predictors = list(X.columns)
predictors

['CREDIT_TO_ANNUITY_RATIO',
 'EXT_SOURCES_MEAN',
 'EXT_SOURCE_3',
 'EXT_SOURCES_MIN',
 'BUREAU_CONSUMER_DAYS_CREDIT_ENDDATE_MAX',
 'AMT_ANNUITY',
 'CREDIT_TO_GOODS_RATIO',
 'DAYS_ID_PUBLISH',
 'DAYS_EMPLOYED',
 'BUREAU_CREDIT_DEBT_CREDIT_DIFF_MEAN',
 'BUREAU_ACTIVE_DEBT_CREDIT_DIFF_MEAN',
 'ANNUITY_TO_INCOME_RATIO',
 'AGE',
 'ORGANIZATION_TYPE',
 'OCCUPATION_TYPE']

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 15 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   CREDIT_TO_ANNUITY_RATIO                  30000 non-null  float64 
 1   EXT_SOURCES_MEAN                         29977 non-null  float64 
 2   EXT_SOURCE_3                             24078 non-null  float64 
 3   EXT_SOURCES_MIN                          29977 non-null  float64 
 4   BUREAU_CONSUMER_DAYS_CREDIT_ENDDATE_MAX  24064 non-null  float64 
 5   AMT_ANNUITY                              30000 non-null  float64 
 6   CREDIT_TO_GOODS_RATIO                    29976 non-null  float64 
 7   DAYS_ID_PUBLISH                          30000 non-null  int64   
 8   DAYS_EMPLOYED                            24577 non-null  float64 
 9   BUREAU_CREDIT_DEBT_CREDIT_DIFF_MEAN      15663 non-null  float64 
 10  BUREAU_ACTIVE_DEBT_CREDIT_DIFF_MEA

## Run A Study

In [13]:
# Nécessite l'installation optuna-integration
def objective(predictors, cat_features=None, optimize_boosting_type=True):
    def _objective(trial):
        if optimize_boosting_type:
            boosting_type = trial.suggest_categorical("boosting_type", ["dart", "gbdt"])
        else:
            boosting_type = "gbdt"
        lambda_l1 = trial.suggest_float(
            'lambda_l1', 1e-8, 10.0, log=True),
        lambda_l2 = trial.suggest_float(
            'lambda_l2', 1e-8, 10.0, log=True),
        num_leaves = trial.suggest_int(
            'num_leaves', 2, 256),
        feature_fraction = trial.suggest_float(
            'feature_fraction', 0.4, 1.0),
        bagging_fraction = trial.suggest_float(
            'bagging_fraction', 0.4, 1.0),
        bagging_freq = trial.suggest_int(
            'bagging_freq', 1, 7),
        min_child_samples = trial.suggest_int(
            'min_child_samples', 5, 100),
        learning_rate = trial.suggest_float(
            "learning_rate", 0.0001, 0.5, log=True),
        max_bin = trial.suggest_int(
            "max_bin", 128, 512, step=32)
        n_estimators = trial.suggest_int(
            "n_estimators", 40, 400, step=20)

        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "binary")
        #pruning_callback = optuna-integration.LightGBMPruningCallback(trial, "binary")
        
        if cat_features:
            model = lgb.LGBMClassifier(
                force_row_wise=True,
                boosting_type=boosting_type,
                n_estimators=n_estimators,
                lambda_l1=lambda_l1,
                lambda_l2=lambda_l2,
                num_leaves=num_leaves,
                feature_fraction=feature_fraction,
                bagging_fraction=bagging_fraction,
                bagging_freq=bagging_freq,
                min_child_samples=min_child_samples,
                learning_rate=learning_rate,
                max_bin=max_bin,
                callbacks=[pruning_callback],
                #feature_name=predictors,
                categorical_feature=cat_features,
                verbose=-1,
                )
        else:
            model = lgb.LGBMClassifier(
                force_row_wise=True,
                boosting_type=boosting_type,
                n_estimators=n_estimators,
                lambda_l1=lambda_l1,
                lambda_l2=lambda_l2,
                num_leaves=num_leaves,
                feature_fraction=feature_fraction,
                bagging_fraction=bagging_fraction,
                bagging_freq=bagging_freq,
                min_child_samples=min_child_samples,
                learning_rate=learning_rate,
                max_bin=max_bin,
                callbacks=[pruning_callback],
                #feature_name=predictors,
                verbose=-1,
                )
        scores = cross_val_score(model, X, y, scoring="f1_macro", cv=5)
        return scores.mean()

    return _objective

In [14]:
cat_features

['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']

In [15]:
with timer("Optimize hyperparameters"):
    sampler = optuna.samplers.TPESampler()
    pruner = optuna.pruners.HyperbandPruner(
        min_resource=10, max_resource=400, reduction_factor=3)

    study = optuna.create_study(
        direction='maximize', sampler=sampler,
        pruner=pruner
    )
    study.optimize(objective(predictors=predictors, cat_features=cat_features), n_trials=30, gc_after_trial=True, n_jobs=-1)

[I 2024-04-26 07:18:28,311] A new study created in memory with name: no-name-e93d7651-4e07-42ff-b967-7f6c3d23a18e
e:\Mes Documents\_Open Classroom\Code\p7\p7env\lib\site-packages\lightgbm\basic.py:2054: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
e:\Mes Documents\_Open Classroom\Code\p7\p7env\lib\site-packages\lightgbm\basic.py:2054: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
e:\Mes Documents\_Open Classroom\Code\p7\p7env\lib\site-packages\lightgbm\basic.py:2034: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
e:\Mes Documents\_Open Classroom\Code\p7\p7env\lib\site-packages\lightgbm\basic.py:2034: UserWarning: categorical

Optimize hyperparameters - done in 79s


In [16]:
print(study.best_trial)
best_params = study.best_trial.params
for k, v in best_params.items():
    print(f"{k} : {v}")

FrozenTrial(number=23, state=TrialState.COMPLETE, values=[0.5907814807779238], datetime_start=datetime.datetime(2024, 4, 26, 7, 18, 28, 352868), datetime_complete=datetime.datetime(2024, 4, 26, 7, 19, 27, 743267), params={'boosting_type': 'dart', 'lambda_l1': 0.025780771002814715, 'lambda_l2': 2.1103357195834125, 'num_leaves': 72, 'feature_fraction': 0.746310813407126, 'bagging_fraction': 0.9601299271053441, 'bagging_freq': 6, 'min_child_samples': 44, 'learning_rate': 0.043556425276861634, 'max_bin': 256, 'n_estimators': 320}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'boosting_type': CategoricalDistribution(choices=('dart', 'gbdt')), 'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_fraction': FloatDistri

## Understanding Parameters

In [18]:
import plotly

In [19]:
plotly.__version__

'5.21.0'

In [13]:
import kaleido
kaleido.__version__

'0.1.0'

In [14]:
study = joblib.load(os.path.join(MODEL_DIR,"lgbm-optuna-study.pkl"))

In [30]:
# Nécessite l'installation de plotly et de Kaleido (redémarrer le kernel impérativement après plotly et kaleido)
# Aparremment il faut kaleido 0.1.0
# essayer pip install "kaleido==0.1.0.post1"
fig = optuna.visualization.plot_parallel_coordinate(study, params=["boosting_type", "num_leaves", "learning_rate", "n_estimators"])
print(type(fig))

<class 'plotly.graph_objs._figure.Figure'>


In [33]:
fig.write_html(os.path.join(MODEL_DIR, 'fig_plotly.html'))

In [32]:
with open("figure.png", "wb") as f:
    fig.write_image(f, format="png", scale=6)

ValueError: 
The kaleido executable is required by the kaleido Python library, but it was not included
in the Python package and it could not be found on the system PATH.

Searched for included kaleido executable at:
    e:\Mes Documents\_Open Classroom\Code\p7\p7env\lib\site-packages\kaleido\executable\kaleido 

Searched for executable 'kaleido' on the following system PATH:
    e:\Mes Documents\_Open Classroom\Code\p7\p7env\Scripts
    E:\Mes Documents\_Open Classroom\Code\p7\p7env\Scripts
    E:\Program Files\Python310\Scripts\
    E:\Program Files\Python310\
    E:\Program Files (x86)\VMware\VMware Player\bin\
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common
    C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR
    E:\Program Files\Git\cmd
    C:\Users\Ariane\AppData\Local\Programs\Python\Python39\Scripts\
    C:\Users\Arian

In [29]:
#fig.write_image(file=os.path.join(MODEL_DIR, "try1_parallel_coordinates.png"), format="png", scale=6)
fig.write_image(file=os.path.join(MODEL_DIR, "try1_parallel_coordinates.jpg"), format="jpg", scale=6)
fig.show()

ValueError: 
The kaleido executable is required by the kaleido Python library, but it was not included
in the Python package and it could not be found on the system PATH.

Searched for included kaleido executable at:
    e:\Mes Documents\_Open Classroom\Code\p7\p7env\lib\site-packages\kaleido\executable\kaleido 

Searched for executable 'kaleido' on the following system PATH:
    e:\Mes Documents\_Open Classroom\Code\p7\p7env\Scripts
    E:\Mes Documents\_Open Classroom\Code\p7\p7env\Scripts
    E:\Program Files\Python310\Scripts\
    E:\Program Files\Python310\
    E:\Program Files (x86)\VMware\VMware Player\bin\
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common
    C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR
    E:\Program Files\Git\cmd
    C:\Users\Ariane\AppData\Local\Programs\Python\Python39\Scripts\
    C:\Users\Arian

In [25]:
#plotly.io.write_image(fig, file=os.path.join(MODEL_DIR, "try1_parallel_coordinates.png"), format="png", scale=6, width=None, height=None, validate=True, engine='auto')
plotly.io.write_image(fig, file="try1_parallel_coordinates.png", format="png", scale=6, width=None, height=None, validate=True, engine='auto')

ValueError: 
The kaleido executable is required by the kaleido Python library, but it was not included
in the Python package and it could not be found on the system PATH.

Searched for included kaleido executable at:
    e:\Mes Documents\_Open Classroom\Code\p7\p7env\lib\site-packages\kaleido\executable\kaleido 

Searched for executable 'kaleido' on the following system PATH:
    e:\Mes Documents\_Open Classroom\Code\p7\p7env\Scripts
    E:\Mes Documents\_Open Classroom\Code\p7\p7env\Scripts
    E:\Program Files\Python310\Scripts\
    E:\Program Files\Python310\
    E:\Program Files (x86)\VMware\VMware Player\bin\
    C:\Windows\system32
    C:\Windows
    C:\Windows\System32\Wbem
    C:\Windows\System32\WindowsPowerShell\v1.0\
    C:\Windows\System32\OpenSSH\
    C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common
    C:\Program Files\NVIDIA Corporation\NVIDIA NvDLISR
    E:\Program Files\Git\cmd
    C:\Users\Ariane\AppData\Local\Programs\Python\Python39\Scripts\
    C:\Users\Arian

In [79]:
fig = optuna.visualization.plot_param_importances(study)
fig.write_image(file=os.path.join(MODEL_DIR, "try1_hyperparam_importance.png"), format="png", scale=6)
fig.show()

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

## Save and Resume a Study

In [23]:
joblib.dump(study, os.path.join(MODEL_DIR, "lgbm-optuna-study.pkl"))

['models/lgbm-optuna-study.pkl']

In [24]:
study = joblib.load("lgbm-optuna-study.pkl")
study.optimize(objective(), n_trials=20, gc_after_trial=True, n_jobs=-1)

FileNotFoundError: [Errno 2] No such file or directory: 'lgbm-optuna-study.pkl'

In [13]:
print(study.best_trial)

FrozenTrial(number=113, state=TrialState.COMPLETE, values=[0.7325947131542604], datetime_start=datetime.datetime(2023, 4, 26, 21, 51, 41, 700790), datetime_complete=datetime.datetime(2023, 4, 26, 21, 51, 58, 269510), params={'boosting_type': 'gbdt', 'lambda_l1': 8.761930063313846e-06, 'lambda_l2': 3.674098080906251e-06, 'num_leaves': 2, 'feature_fraction': 0.8013834287179691, 'bagging_fraction': 0.8930729866672722, 'bagging_freq': 1, 'min_child_samples': 68, 'learning_rate': 0.14747560109267427, 'max_bin': 416, 'n_estimators': 140}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'boosting_type': CategoricalDistribution(choices=('dart', 'gbdt')), 'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_fraction': Float

## Save and Resume on a remote DB

In [21]:
sampler = optuna.samplers.TPESampler()
pruner = optuna.pruners.HyperbandPruner(min_resource=10, max_resource=400, reduction_factor=3)

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "lgbm-tpe-rdb-study"
storage_name = f"sqlite:///{study_name}.db"
study = optuna.create_study(
    study_name=study_name,
    storage=storage_name,
    load_if_exists=False,
    sampler=sampler,
    pruner=pruner)
study.optimize(objective(False), n_trials=100, gc_after_trial=True, n_jobs=-1)

[I 2023-04-26 21:55:54,747] A new study created in RDB with name: lgbm-cma-es-rdb-study


A new study created in RDB with name: lgbm-cma-es-rdb-study


[I 2023-04-26 21:56:33,779] Trial 3 finished with value: 0.7132657507447453 and parameters: {'lambda_l1': 2.2532403757797748e-05, 'lambda_l2': 0.05662631493366982, 'num_leaves': 91, 'feature_fraction': 0.8673704694022113, 'bagging_fraction': 0.7408227257890989, 'bagging_freq': 3, 'min_child_samples': 69, 'learning_rate': 0.20682759099882317, 'max_bin': 512, 'n_estimators': 60}. Best is trial 3 with value: 0.7132657507447453.


Trial 3 finished with value: 0.7132657507447453 and parameters: {'lambda_l1': 2.2532403757797748e-05, 'lambda_l2': 0.05662631493366982, 'num_leaves': 91, 'feature_fraction': 0.8673704694022113, 'bagging_fraction': 0.7408227257890989, 'bagging_freq': 3, 'min_child_samples': 69, 'learning_rate': 0.20682759099882317, 'max_bin': 512, 'n_estimators': 60}. Best is trial 3 with value: 0.7132657507447453.


[I 2023-04-26 21:56:34,880] Trial 4 finished with value: 0.7226552433871103 and parameters: {'lambda_l1': 0.006182106804178753, 'lambda_l2': 6.160625321836477, 'num_leaves': 77, 'feature_fraction': 0.941323475749757, 'bagging_fraction': 0.4142484304524909, 'bagging_freq': 6, 'min_child_samples': 94, 'learning_rate': 0.017326662146954618, 'max_bin': 512, 'n_estimators': 180}. Best is trial 3 with value: 0.7132657507447453.


...

[I 2023-04-26 22:02:59,748] Trial 50 finished with value: 0.4235082247366586 and parameters: {'lambda_l1': 0.0012739682245968972, 'lambda_l2': 9.842818164648099e-05, 'num_leaves': 147, 'feature_fraction': 0.5631666822474554, 'bagging_fraction': 0.7274109201998455, 'bagging_freq': 5, 'min_child_samples': 53, 'learning_rate': 0.0022965647473676305, 'max_bin': 416, 'n_estimators': 180}. Best is trial 5 with value: 0.4235082247366586.


Trial 50 finished with value: 0.4235082247366586 and parameters: {'lambda_l1': 0.0012739682245968972, 'lambda_l2': 9.842818164648099e-05, 'num_leaves': 147, 'feature_fraction': 0.5631666822474554, 'bagging_fraction': 0.7274109201998455, 'bagging_freq': 5, 'min_child_samples': 53, 'learning_rate': 0.0022965647473676305, 'max_bin': 416, 'n_estimators': 180}. Best is trial 5 with value: 0.4235082247366586.


[I 2023-04-26 22:03:00,457] Trial 49 finished with value: 0.4235082247366586 and parameters: {'lambda_l1': 0.0050038474922444315, 'lambda_l2': 5.162993438119722e-06, 'num_leaves': 103, 'feature_fraction': 0.5098782589132528, 'bagging_fraction': 0.677029585535416, 'bagging_freq': 3, 'min_child_samples': 52, 'learning_rate': 0.00020630631963538095, 'max_bin': 448, 'n_estimators': 220}. Best is trial 5 with value: 0.4235082247366586.


Trial 49 finished with value: 0.4235082247366586 and parameters: {'lambda_l1': 0.0050038474922444315, 'lambda_l2': 5.162993438119722e-06, 'num_leaves': 103, 'feature_fraction': 0.5098782589132528, 'bagging_fraction': 0.677029585535416, 'bagging_freq': 3, 'min_child_samples': 52, 'learning_rate': 0.00020630631963538095, 'max_bin': 448, 'n_estimators': 220}. Best is trial 5 with value: 0.4235082247366586.


RuntimeError: dictionary changed size during iteration

In [12]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective(), n_trials=20, gc_after_trial=True, n_jobs=-1)

[I 2023-04-21 08:25:32,162] Using an existing study with name 'lgbm-optuna-rdb-study' instead of creating a new one.


Using an existing study with name 'lgbm-optuna-rdb-study' instead of creating a new one.


[I 2023-04-21 08:26:17,345] Trial 16 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.0179602799796386e-08, 'lambda_l2': 9.349435489679461, 'num_leaves': 247, 'feature_fraction': 0.7460125770492829, 'bagging_fraction': 0.8227537764241121, 'bagging_freq': 7, 'min_child_samples': 98, 'learning_rate': 0.00020887902150769306, 'max_bin': 512, 'n_estimators': 140}. Best is trial 2 with value: 0.4235082247366586.


Trial 16 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.0179602799796386e-08, 'lambda_l2': 9.349435489679461, 'num_leaves': 247, 'feature_fraction': 0.7460125770492829, 'bagging_fraction': 0.8227537764241121, 'bagging_freq': 7, 'min_child_samples': 98, 'learning_rate': 0.00020887902150769306, 'max_bin': 512, 'n_estimators': 140}. Best is trial 2 with value: 0.4235082247366586.


[I 2023-04-21 08:26:17,706] Trial 10 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.1909111890008977e-08, 'lambda_l2': 3.916792214730818, 'num_leaves': 252, 'feature_fraction': 0.7842462593069699, 'bagging_fraction': 0.8275752521380275, 'bagging_freq': 7, 'min_child_samples': 100, 'learning_rate': 0.00011984328761926979, 'max_bin': 512, 'n_estimators': 140}. Best is trial 2 with value: 0.4235082247366586.


...

Trial 14 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'gbdt', 'lambda_l1': 1.4408133646747365e-08, 'lambda_l2': 2.837493258342436, 'num_leaves': 245, 'feature_fraction': 0.7637633482746569, 'bagging_fraction': 0.8296627735915492, 'bagging_freq': 7, 'min_child_samples': 37, 'learning_rate': 0.00013694284525827822, 'max_bin': 512, 'n_estimators': 160}. Best is trial 2 with value: 0.4235082247366586.


[I 2023-04-21 08:26:34,243] Trial 19 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'dart', 'lambda_l1': 1.3957343825730509e-06, 'lambda_l2': 1.2256972812369361e-08, 'num_leaves': 5, 'feature_fraction': 0.8427033256168555, 'bagging_fraction': 0.5366577275595632, 'bagging_freq': 1, 'min_child_samples': 36, 'learning_rate': 0.0007048212563581564, 'max_bin': 224, 'n_estimators': 40}. Best is trial 2 with value: 0.4235082247366586.


Trial 19 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'dart', 'lambda_l1': 1.3957343825730509e-06, 'lambda_l2': 1.2256972812369361e-08, 'num_leaves': 5, 'feature_fraction': 0.8427033256168555, 'bagging_fraction': 0.5366577275595632, 'bagging_freq': 1, 'min_child_samples': 36, 'learning_rate': 0.0007048212563581564, 'max_bin': 224, 'n_estimators': 40}. Best is trial 2 with value: 0.4235082247366586.


[I 2023-04-21 08:26:34,762] Trial 18 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'dart', 'lambda_l1': 1.2876121235965678e-06, 'lambda_l2': 1.542472294392896e-07, 'num_leaves': 4, 'feature_fraction': 0.8472764387650585, 'bagging_fraction': 0.5063448986999544, 'bagging_freq': 1, 'min_child_samples': 34, 'learning_rate': 0.0005789551103378422, 'max_bin': 224, 'n_estimators': 40}. Best is trial 2 with value: 0.4235082247366586.


Trial 18 finished with value: 0.4235082247366586 and parameters: {'boosting_type': 'dart', 'lambda_l1': 1.2876121235965678e-06, 'lambda_l2': 1.542472294392896e-07, 'num_leaves': 4, 'feature_fraction': 0.8472764387650585, 'bagging_fraction': 0.5063448986999544, 'bagging_freq': 1, 'min_child_samples': 34, 'learning_rate': 0.0005789551103378422, 'max_bin': 224, 'n_estimators': 40}. Best is trial 2 with value: 0.4235082247366586.


In [23]:
print(study.best_trial)

FrozenTrial(number=5, state=TrialState.COMPLETE, values=[0.4235082247366586], datetime_start=datetime.datetime(2023, 4, 26, 21, 55, 55, 15345), datetime_complete=datetime.datetime(2023, 4, 26, 21, 56, 38, 244296), params={'bagging_fraction': 0.9277029430467971, 'bagging_freq': 3, 'feature_fraction': 0.6565211338929926, 'lambda_l1': 4.496791174062579, 'lambda_l2': 0.000612013760106219, 'learning_rate': 0.00011283703507767392, 'max_bin': 288, 'min_child_samples': 97, 'n_estimators': 180, 'num_leaves': 100}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'bagging_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_freq': IntDistribution(high=7, log=False, low=1, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'learning_rate': FloatDistribution(high=0

## Multi-objective optimization

In [22]:
def moo_objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 0.0001, 0.5, log=True),

    model = lgb.LGBMClassifier(
        force_row_wise=True,
        boosting_type='gbdt',
        n_estimators=200,
        lambda_l1=3.298803078077973e-07,
        lambda_l2=8.938532783741386e-07,
        num_leaves=6,
        feature_fraction=0.5133218336120866,
        bagging_fraction=0.9660809666082303,
        bagging_freq=7,
        min_child_samples=91,
        learning_rate=learning_rate,
        max_bin=320,
        verbose=-1,
    )
    scores = cross_val_score(model, X, y, scoring="f1_macro")
    return learning_rate[0], scores.mean()

In [23]:
study = optuna.create_study(directions=["maximize", "maximize"])
study.optimize(moo_objective, n_trials=100)

[I 2023-04-27 10:47:35,501] A new study created in memory with name: no-name-eb122531-f2ee-4cb5-92d1-8efba6a28eef
[I 2023-04-27 10:47:36,221] Trial 0 finished with values: [0.018276625572235056, 0.7229728382357858] and parameters: {'learning_rate': 0.018276625572235056}. 
[I 2023-04-27 10:47:36,922] Trial 1 finished with values: [0.012281350875017864, 0.713129498994182] and parameters: {'learning_rate': 0.012281350875017864}. 
[I 2023-04-27 10:47:37,601] Trial 2 finished with values: [0.3179931543219117, 0.7181352358216049] and parameters: {'learning_rate': 0.3179931543219117}. 
[I 2023-04-27 10:47:38,317] Trial 3 finished with values: [0.3317048028124761, 0.7182462105095208] and parameters: {'learning_rate': 0.3317048028124761}. 
[I 2023-04-27 10:47:39,042] Trial 4 finished with values: [0.010097499163932589, 0.689595604293249] and parameters: {'learning_rate': 0.010097499163932589}. 
[I 2023-04-27 10:47:39,716] Trial 5 finished with values: [0.0010284892903758126, 0.4235082247366586]

In [24]:
fig = optuna.visualization.plot_pareto_front(study, target_names=["learning_rate", "f1"])
fig.write_image(file="figures/ch5_pareto.png", format="png", scale=6)
fig.show()